In [ ]:
# Load in Python libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc

In [ ]:
# Retrieve Data
file = "stbly/TKE_terms_height_63.213952337529214_interior_velocity_0.1_visc_1.0e-5_Sinf_1.0000287024749517_gamma_0.500235593669851_theta_1.8113_f_0.0001_N2_1.0e-5.nc"

ocean_data=xr.open_dataset("/glade/derecho/scratch/knudsenl/data/new_data/"+file).chunk(time=1)#.isel(time=slice(0,4000)) #i=103

In [ ]:
# Set parameters for simulation
theta = 1.8113
N2 = 1e-5
Vinf1 = 0.1
Vinf = str(Vinf1)
f = 10**(-4)
fstar = np.cos(theta*np.pi/180)*(1e-8+np.tan(theta*np.pi/180)**2*N2)**(0.5)

In [ ]:
# Calculate the TKE
KE_spatial_average_time_series = ocean_data.k 
inertial_period = (KE_spatial_average_time_series.time/pd.Timedelta("1s"))*(fstar/(2*np.pi))
KE_spatial_average_time_series_ip = KE_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))
seconds = KE_spatial_average_time_series.time/pd.Timedelta("1s")

In [ ]:
# Plot the TKE
plt.plot(KE_spatial_average_time_series_ip.intp[:],KE_spatial_average_time_series_ip.values[:]/KE_spatial_average_time_series_ip.values[0])
plt.title("Perturbation kinetic energy over time")
plt.ylabel('$k$/$k_5$')
plt.xlabel('$tf^*/2\pi$')
plt.yscale('log')
# plt.savefig("PKEplot.png")
plt.show()

In [ ]:
# Find the linear phase
ts = 1000
tf = 3000
plt.plot(KE_spatial_average_time_series_ip.intp[ts:tf],KE_spatial_average_time_series_ip.values[ts:tf]/KE_spatial_average_time_series_ip.values[ts])
# plt.title("Pertubation Kinetic Energy During Linear Phase")
plt.ylabel('$k$/$k_{10}$')
plt.xlabel('$tf^*/(2\pi)$')
plt.yscale('log')
# plt.savefig("linear_phase_plot.png")
plt.show()

In [ ]:
# Perform linear regression on the data
lr = sc.stats.linregress(x=seconds[ts:tf],y=np.log(KE_spatial_average_time_series_ip.values[ts:tf]/KE_spatial_average_time_series_ip.values[ts]))
lr

In [ ]:
# State the growth rate
grwthrate = lr.slope
grwthrate = np.round(grwthrate,7)
grwthrt = str(grwthrate/2)
print(f"The growth rate of the linear phase is {grwthrate} $m^2 s^-3$")

In [ ]:
# Plot the linear regression to confirm
plt.plot(KE_spatial_average_time_series_ip.intp[ts:tf],KE_spatial_average_time_series_ip.values[ts:tf]/KE_spatial_average_time_series_ip.values[ts],label="TKE")
plt.plot(seconds[ts:tf]*fstar/(2*np.pi),np.exp(lr.slope*seconds[ts:tf]+lr.intercept),label="Linear Regression Fit")
plt.title(r"Linear Phase: Growth Rate  "+grwthrt+" $m^2 s^{-3}$")
plt.ylabel('$k$/$k_{5}$')
plt.xlabel('$t\dfrac{f^*}{2\pi}$')
plt.legend()
plt.yscale('log')
plt.show()

Calculate the TKE terms below

In [ ]:
AGSP_spatial_average_time_series = ocean_data.AGSP # .sel(zC=np.arange(0,h+dh,dh),method="nearest")
inertial_period = AGSP_spatial_average_time_series.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
AGSP_spatial_average_time_series_ip = AGSP_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))


In [ ]:
WSP_spatial_average_time_series = ocean_data.WSP #.mean(dim=['xC','yC','zC']) # sel(zC=np.arange(0,h+dh,dh),method="nearest")
inertial_period = WSP_spatial_average_time_series.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
WSP_spatial_average_time_series_ip = WSP_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))

In [ ]:
GSP_spatial_average_time_series = ocean_data.GSP #.mean(dim=['xC','yC','zC']) # sel(zC=np.arange(0,h+dh,dh),method="nearest").
inertial_period = GSP_spatial_average_time_series.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
GSP_spatial_average_time_series_ip = GSP_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))


In [ ]:
BFLUX_spatial_average_time_series = ocean_data.BFLUX #.mean(dim=['xC','yC','zC']) # .sel(zC=np.arange(0,h+dh,dh),method="nearest")
inertial_period = BFLUX_spatial_average_time_series.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
BFLUX_spatial_average_time_series_ip = BFLUX_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))


In [ ]:
E_spatial_average_time_series = ocean_data.E #.mean(dim=['xC','yC','zC']) # .sel(zC=np.arange(0,h+dh,dh),method="nearest")
inertial_period = E_spatial_average_time_series.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
E_spatial_average_time_series_ip = E_spatial_average_time_series.assign_coords(intp = ('time',inertial_period.data))


In [ ]:
KE_turb = ocean_data.k #.mean(dim=['xC','yC','zC']) # .sel(zC=np.arange(0,h+dh,dh)
seconds = KE_turb.time/pd.Timedelta("1s")
inertial_period = KE_turb.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
KE_turb_ip = KE_turb.assign_coords(intp = ('time',inertial_period.data))
KE_turb_ip = KE_turb_ip.assign_coords(sec = ('time',seconds.data))
dkdt = KE_turb_ip.differentiate(coord='sec',edge_order=-1)

In [ ]:
# Plot the TKE terms over a single inertial period normalized to TKE
ts2 = 2000
tf2 = 2801
sum = dkdt[ts2:tf2]-AGSP_spatial_average_time_series_ip.values[ts2:tf2]-WSP_spatial_average_time_series_ip.values[ts2:tf2]-GSP_spatial_average_time_series_ip.values[ts2:tf2]-BFLUX_spatial_average_time_series_ip.values[ts2:tf2] +E_spatial_average_time_series_ip.values[ts2:tf2] #+PWORK_spatial_average_time_series_ip.values[ts2:tf2] #+KTRANS_spatial_average_time_series_ip.values[ts2:tf2]+PWORK_spatial_average_time_series_ip.values[ts2:tf2]
plt.plot(dkdt.intp[ts2:tf2],dkdt[ts2:tf2]/(KE_turb_ip[ts2:tf2]*fstar),color="k",label='$\dfrac{1}{k}\dfrac{\partial k}{\partial t}$')
plt.plot(AGSP_spatial_average_time_series_ip.intp[ts2:tf2],AGSP_spatial_average_time_series_ip.values[ts2:tf2]/(KE_turb_ip.values[ts2:tf2]*fstar),color='red',label='AGSP/k')
plt.plot(GSP_spatial_average_time_series_ip.intp[ts2:tf2],GSP_spatial_average_time_series_ip.values[ts2:tf2]/(KE_turb_ip.values[ts2:tf2]*fstar),color='green',label='GSP/k')
plt.plot(WSP_spatial_average_time_series_ip.intp[ts2:tf2],WSP_spatial_average_time_series_ip.values[ts2:tf2]/(KE_turb_ip.values[ts2:tf2]*fstar),color='cyan',label='WSP/k')
plt.plot(BFLUX_spatial_average_time_series_ip.intp[ts2:tf2],BFLUX_spatial_average_time_series_ip.values[ts2:tf2]/(KE_turb_ip.values[ts2:tf2]*fstar),color='blue',label='BFLUX/k')
plt.plot(E_spatial_average_time_series_ip.intp[ts2:tf2],-1*E_spatial_average_time_series_ip.values[ts2:tf2]/(KE_turb_ip.values[ts2:tf2]*fstar),color='pink',label='$-\epsilon/k$')
plt.plot(dkdt.intp[ts2:tf2],sum.values/(KE_turb_ip[ts2:tf2]*fstar),color="k",linestyle="--", label="Residual/k")
plt.ylabel("TKE growth rate/$f^*$")
plt.legend(loc="lower left",prop={'size':9},ncol=4,handlelength=1) 
plt.show()

In [ ]:
# Calculate non-normalized TKE
ts = 2000
tf = 2801 
plt.plot(dkdt.intp[ts:tf],dkdt[ts:tf],color="k",label='$\dfrac{\partial k}{\partial t}$')
plt.plot(AGSP_spatial_average_time_series_ip.intp[ts:tf],AGSP_spatial_average_time_series_ip.values[ts:tf],color='red',label='AGSP')
plt.plot(GSP_spatial_average_time_series_ip.intp[ts:tf],GSP_spatial_average_time_series_ip.values[ts:tf],color='green',label='GSP')
plt.plot(WSP_spatial_average_time_series_ip.intp[ts:tf],WSP_spatial_average_time_series_ip.values[ts:tf],color='cyan',label='WSP')
plt.plot(BFLUX_spatial_average_time_series_ip.intp[ts:tf],BFLUX_spatial_average_time_series_ip.values[ts:tf],color='blue',label='BFLUX')
# plt.plot(BFLUX_spatial_average_time_series_ip.intp[ts:tf],BFLUX_spatial_average_time_series_ip.values[ts:tf]+GSP_spatial_average_time_series_ip.values[ts:tf],color='grey',label='BFLUX+GSP')
plt.plot(E_spatial_average_time_series_ip.intp[ts:tf],-1*E_spatial_average_time_series_ip.values[ts:tf],color='pink',label='$-\epsilon$')
# plt.plot(DIFF_spatial_average_time_series_ip.intp[ts:tf],-1*DIFF_spatial_average_time_series_ip.values[ts:tf],color='orange',label="DIFF/k")
# plt.plot(E_spatial_average_time_series_ip.intp[ts:tf],-1*E_spatial_average_time_series_ip.values[ts:tf]+dkdt[ts:tf],color='pink',label='$-\epsilon$')
# plt.plot(PWORK_spatial_average_time_series_ip.intp[ts:tf],-1*PWORK_spatial_average_time_series_ip.values[ts:tf],color='orange',label="PWORK/k")
sum = dkdt[ts:tf]-AGSP_spatial_average_time_series_ip.values[ts:tf]-WSP_spatial_average_time_series_ip.values[ts:tf]-GSP_spatial_average_time_series_ip.values[ts:tf]-BFLUX_spatial_average_time_series_ip.values[ts:tf]+E_spatial_average_time_series_ip.values[ts:tf]
plt.plot(dkdt.intp[ts:tf],sum.values,color="k",linestyle="--", label="Residual/k")
# plt.axhline(color="violet")
# plt.yscale("symlog",linthresh=1e-16)
plt.xlabel("$t\dfrac{f^*}{2\pi}$")
plt.ylabel("TKE growth rate")
plt.title("Non-normalized TKE Terms")
plt.legend(loc='lower left')
# plt.yscale("symlog",linthresh=1e-14)
plt.figure(figsize=(12,5))
# plt.savefig("TKE_time_terms.pdf")
plt.show()

In [ ]:
# PLot the non-normalized residual
sum = dkdt[ts:tf]-AGSP_spatial_average_time_series_ip.values[ts:tf]-WSP_spatial_average_time_series_ip.values[ts:tf]-GSP_spatial_average_time_series_ip.values[ts:tf]-BFLUX_spatial_average_time_series_ip.values[ts:tf]+E_spatial_average_time_series_ip.values[ts:tf] #+PWORK_spatial_average_time_series_ip.values[ts:tf]
plt.plot(sum.intp,sum.values,color="k")
plt.axhline(color="violet")
plt.xlabel("$t\dfrac{f^*}{2\pi}$")
plt.ylabel("Residual of the TKE")
plt.title("Residual of TKE Terms")
# plt.legend(loc='lower left')
plt.figure(figsize=(12,5))
# plt.savefig("TKE_time_terms.pdf")
plt.show()

Below we calculate the integrated TKE budget

In [ ]:
seconds = GSP_spatial_average_time_series_ip.time/pd.Timedelta("1s")
AGSP_spatial_average_time_series_ip = AGSP_spatial_average_time_series_ip.assign_coords(sec = ('time',seconds.data))
integrated_AGSP = (AGSP_spatial_average_time_series_ip.isel(time=slice(0,-1))).cumulative_integrate(coord="sec")

In [ ]:
seconds = WSP_spatial_average_time_series_ip.time/pd.Timedelta("1s")
WSP_spatial_average_time_series_ip = WSP_spatial_average_time_series_ip.assign_coords(sec = ('time',seconds.data))
integrated_WSP = (WSP_spatial_average_time_series_ip.isel(time=slice(0,-1))).cumulative_integrate(coord="sec")

In [ ]:
seconds = GSP_spatial_average_time_series_ip.time/pd.Timedelta("1s")
GSP_spatial_average_time_series_ip = GSP_spatial_average_time_series_ip.assign_coords(sec = ('time',seconds.data))
integrated_GSP = (GSP_spatial_average_time_series_ip.isel(time=slice(0,-1))).cumulative_integrate(coord="sec")

In [ ]:
seconds = BFLUX_spatial_average_time_series_ip.time/pd.Timedelta("1s")
BFLUX_spatial_average_time_series_ip = BFLUX_spatial_average_time_series_ip.assign_coords(sec = ('time',seconds.data))
integrated_BFLUX = (BFLUX_spatial_average_time_series_ip.isel(time=slice(0,-1))).cumulative_integrate(coord="sec")

In [ ]:
seconds = E_spatial_average_time_series_ip.time/pd.Timedelta("1s")
E_spatial_average_time_series_ip = E_spatial_average_time_series_ip.assign_coords(sec = ('time',seconds.data))
integrated_E = (-1*E_spatial_average_time_series_ip.isel(time=slice(0,-1))).cumulative_integrate(coord="sec")

In [ ]:
KE_turb = ocean_data.k *((fstar)/(2*np.pi))**(2) #.mean(dim=['xC','yC','zC']) # .sel(zC=np.arange(0,h+dh,dh)
seconds = KE_turb.time/pd.Timedelta("1s")
inertial_period = KE_turb.time/pd.Timedelta("1s")*(fstar)/(2*np.pi)
KE_turb_ip = KE_turb.assign_coords(intp = ('time',inertial_period.data))
KE_turb_ip = KE_turb_ip.assign_coords(sec = ('time',seconds.data))
KE_turb_ip = KE_spatial_average_time_series_ip-KE_spatial_average_time_series_ip.isel(time=0)

In [ ]:
# Plot the integrated TKE terms
start = 0 
end = -1
sum = KE_spatial_average_time_series_ip-integrated_AGSP-integrated_GSP-integrated_WSP-integrated_BFLUX-integrated_E #-integrated_DIFF
plt.figure(figsize=(7,5))
plt.plot(KE_spatial_average_time_series_ip.intp[start:end],KE_spatial_average_time_series_ip[start:end]-KE_spatial_average_time_series_ip[start],label="$k$",color="k")
plt.plot(integrated_AGSP.intp[start:end],integrated_AGSP[start:end]-integrated_AGSP[start],label="$\int_0^tASP\,dt'$",color="red")
plt.plot(integrated_GSP.intp[start:end],integrated_GSP[start:end]-integrated_GSP[start],label="$\int_0^tGSP\,dt'$",color="green")
plt.plot(integrated_WSP.intp[start:end],integrated_WSP[start:end]-integrated_WSP[start],label="$\int_0^tWSP\,dt'$",color="cyan")
plt.plot(integrated_BFLUX.intp[start:end],integrated_BFLUX[start:end]-integrated_BFLUX[start],label="$\int_0^tBP\,dt'$",color="blue")
plt.plot(integrated_E.intp[start:end],integrated_E[start:end]-integrated_E[start],label="$-\int_0^t\epsilon\,dt'$",color="pink")
plt.plot(sum.intp[start:end],sum[start:end]-sum[start],label="Residual",color="grey",alpha=0.9)
plt.xlabel("Modified Inertial Period")
plt.ylabel(r"k $(m^2 s^{-2})$")
plt.yscale("symlog",linthresh=1e-16)
# plt.yticks([1e-12,-1e-12,1e-10,-1e-10,1e-8,-1e-8,1e-6,-1e-6,1e-4,-1e-4])
plt.legend(loc="lower left",prop={'size':7.5},ncol=2,handlelength=0.5)
plt.xlim([KE_spatial_average_time_series_ip.intp[start]-0.01,KE_spatial_average_time_series_ip.intp[end]+0.01])
# plt.ylim([-5e-4,5e-4])

# plt.savefig("S_1_0_gamma_l_delta_0_5_time_integrated_TKE_terms_linthresh_1_e_12.pdf")

plt.show()

In [ ]:
# Plot the residual
plt.plot(sum.intp,sum)
plt.title("Residual of Turbulent Kinetic Energy")
plt.xlabel("$t\dfrac{f^*}{2\pi}$")
plt.ylabel("Residual")